# Week 6 Homework

***Due (pushed to your GitHub branch) on 10/24 by 11:59 pm***

## STAR two-pass alignment

Build the STAR SA:

In [1]:
!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir ../2_genome_exploration/genome/star \
--genomeFastaFiles ../2_genome_exploration/genome/zebrafish_genome.fa \
--sjdbGTFfile ../2_genome_exploration/genome/zebrafish_annotations.gtf \
--sjdbOverhang 84 \
--genomeSAindexNbases 13

	STAR --runThreadN 32 --runMode genomeGenerate --genomeDir ../2_genome_exploration/genome/star --genomeFastaFiles ../2_genome_exploration/genome/zebrafish_genome.fa --sjdbGTFfile ../2_genome_exploration/genome/zebrafish_annotations.gtf --sjdbOverhang 84 --genomeSAindexNbases 13
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 20 21:26:15 ..... started STAR run
Oct 20 21:26:15 ... starting to generate Genome files
Oct 20 21:26:42 ..... processing annotations GTF
Oct 20 21:26:58 ... starting to sort Suffix Array. This may take a long time...
Oct 20 21:27:06 ... sorting Suffix Array chunks and saving them to disk...
Oct 20 21:29:37 ... loading chunks from disk, packing SA...
Oct 20 21:31:19 ... finished generating suffix array
Oct 20 21:31:19 ... generating Suffix Array index
Oct 20 21:32:59 ... completed Suffix Array index
Oct 20 21:33:00 ..... inserting junctions into the genome indices
Oct 20 21:34:16 ... writing Genome

Use STAR and ***all*** of your FASTQs for a first-pass alignment. Be sure to update the `sjdbOverhang` parameter to a value that is appropriate for your dataset. Rename the log file so that you can compare the logs from the first and second alignments:

In [2]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/genome/star \
--readFilesManifest hwmanifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM Unsorted \
--outSAMunmapped Within \
--outFileNamePrefix alignment/star/

!mv alignment/star/Log.final.out alignment/star/hwfirst-pass.final.out

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/genome/star --readFilesManifest hwmanifest.tsv --readFilesCommand zcat --outSAMtype BAM Unsorted --outSAMunmapped Within --outFileNamePrefix alignment/star/
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 20 21:39:42 ..... started STAR run
Oct 20 21:39:43 ..... loading genome
Oct 20 21:40:34 ..... started mapping
Oct 20 21:43:54 ..... finished mapping
Oct 20 21:43:59 ..... finished successfully


Use STAR, all of your FASTQs, and `SJ.out.tab` for the second-pass alignment. You will likely need to make your own `manifest.tsv`:

In [3]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/genome/star \
--readFilesManifest hwmanifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM Unsorted \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment/star/ \
--sjdbFileChrStartEnd alignment/star/SJ.out.tab

!mv alignment/star/Log.final.out alignment/star/hwsecond-pass.final.out

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/genome/star --readFilesManifest hwmanifest.tsv --readFilesCommand zcat --outSAMtype BAM Unsorted --outSAMunmapped Within --outSAMattributes NH HI AS nM RG --outFileNamePrefix alignment/star/ --sjdbFileChrStartEnd alignment/star/SJ.out.tab
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 20 21:45:53 ..... started STAR run
Oct 20 21:45:54 ..... loading genome
Oct 20 21:46:04 ..... inserting junctions into the genome indices
Oct 20 21:46:41 ..... started mapping
Oct 20 21:50:03 ..... finished mapping
Oct 20 21:50:06 ..... finished successfully


In [1]:
!samtools sort -@ 32 -o alignment/star/Aligned.sorted.bam alignment/star/Aligned.out.bam

[bam_sort_core] merging from 0 files and 32 in-memory blocks...


Update this block to briefly describe the differences between the logs for the first and second alignments.

## HISAT alignment

(***Note***: you may need Dr. Wheeler to build the HISAT index of your genome. Ask him for assistance when you get to this part of the assignment.)

Use a Bash `while` loop to align all of your FASTQs and save the output files using the same names as the input FASTQ:

In [4]:
!while read -r line; do \
    fq1=$(echo $line | awk '{print $1}'); \
    fq2=$(echo $line | awk '{print $2}'); \
    bn=$(basename "$fq1" .fastq.gz) \
    bn=$(echo "$bn" | sed 's/_1$//') \
    rg=$(echo $line | awk '{print $3}' | sed 's/ID://' ); \
    echo "Aligning $fq1 and $fq2 with $rg"; \
    hisat2 /data/classes/2025/fall/biol343/course_files/student_genomes/kyle/hisat/zebrafish_genome -p 16 -1 $fq1 -2 $fq2 --rg-id $rg --rg SM:$rg --summary-file alignment/hisat/$bn.log --new-summary > alignment/hisat/$bn.sam; \
    done < hwmanifest.tsv

Aligning /data/users/heky1803/BIOL343/5_fastq/hw_fastq/ERR1442621_1.fastq.gz and /data/users/heky1803/BIOL343/5_fastq/hw_fastq/ERR1442621_2.fastq.gz with DOME_A
HISAT2 summary stats:
	Total pairs: 1706411
		Aligned concordantly or discordantly 0 time: 296369 (17.37%)
		Aligned concordantly 1 time: 1344497 (78.79%)
		Aligned concordantly >1 times: 56956 (3.34%)
		Aligned discordantly 1 time: 8589 (0.50%)
	Total unpaired reads: 592738
		Aligned 0 time: 440950 (74.39%)
		Aligned 1 time: 133825 (22.58%)
		Aligned >1 times: 17963 (3.03%)
	Overall alignment rate: 87.08%
Aligning /data/users/heky1803/BIOL343/5_fastq/hw_fastq/ERR1442622_1.fastq.gz and /data/users/heky1803/BIOL343/5_fastq/hw_fastq/ERR1442622_2.fastq.gz with DOME_B
HISAT2 summary stats:
	Total pairs: 1853271
		Aligned concordantly or discordantly 0 time: 313015 (16.89%)
		Aligned concordantly 1 time: 1468545 (79.24%)
		Aligned concordantly >1 times: 61224 (3.30%)
		Aligned discordantly 1 time: 10487 (0.57%)
	Total unpaired reads

## Snakemake Pipeline

In contrast to last week (FASTQ files), there aren't as many decisions that need to be made during alignment, so using a notebook to control and report code isn't as necessary. As you saw in in [6_alignment](6_alignment.ipynb), file and directory management can get complicated during alignment. This is a case where Snakemake scripting shines.

1. Copy your `Snakefile` and `config.yaml` from `5_fastq` to `6_alignment`. 
2. In the `Snakefile`, add the following rules:  
    a. `fetch_genome` - using a variation from the code from `2_genome_exploration`, download your reference genome and annotations.  
    b. `star_sa` - use `STAR` to produce the suffix array of the reference genome.  
    c. `star_align_first` - use `STAR` to perform first-pass alignment with all FASTQ files.  
    d. `star_align_second` - use `STAR` to perform second-pass alignment with all FASTQ files.  
    e. `hisat_align` - use `HISAT` to align all FASTQ files.  
    f. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.